*   Victor Muryn
*   Kostyantyn Savorona
*   Yurii Sahaidak

In [ ]:
%pip install -q torch transformers accelerate bitsandbytes sentence-transformers faiss-gpu langchain langchain_community peft
%pip install -q git+https://github.com/robinhad/ukrainian-tts.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import time
import pickle

import torch
import numpy as np
from tqdm import tqdm
from google.colab import userdata
import IPython.display as ipd

from langchain.embeddings import HuggingFaceEmbeddings

import faiss

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoProcessor, AutoModelForCTC

from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from peft import PeftModel, PeftConfig

from langchain_core.runnables import RunnablePassthrough

import scipy.io.wavfile as wavfile
import librosa

from ukrainian_tts.tts import TTS, Voices, Stress

In [ ]:
!gdown 1FLg1UlQuH6E7fzIFeiATsawIby48hQs3
!unzip rag-assistant.zip

Downloading...
From: https://drive.google.com/uc?id=1FLg1UlQuH6E7fzIFeiATsawIby48hQs3
To: /content/rag-assistant.zip
100% 1.88M/1.88M [00:00<00:00, 14.5MB/s]
Archive:  rag-assistant.zip
  inflating: embeddings_openai.pickle  
  inflating: __MACOSX/._embeddings_openai.pickle  
  inflating: questions.pickle        
  inflating: __MACOSX/._questions.pickle  
  inflating: modified_questions.json  
  inflating: __MACOSX/._modified_questions.json  


In [ ]:
with open('questions.pickle', 'rb') as handle:
    questions = pickle.load(handle)

questions = np.array(questions)

# Speech to Text

In [ ]:
# Load model directly
processor_stt = AutoProcessor.from_pretrained("robinhad/wav2vec2-xls-r-300m-uk")
model_stt = AutoModelForCTC.from_pretrained("robinhad/wav2vec2-xls-r-300m-uk").to("cuda")

In [ ]:
def process_audio(audio_path: str):
  sampling_rate, data = wavfile.read(audio_path)

  if len(data.shape) > 1:
      data = data.mean(axis=1)

  if sampling_rate != 16000:
      data = librosa.resample(data.astype(float), orig_sr=sampling_rate, target_sr=16000)

  audio = torch.tensor(data).to('cuda')
  input_dict = processor_stt(audio, return_tensors="pt", padding=True)
  logits = model_stt(input_dict.input_values.to("cuda")).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]

  return processor_stt.decode(pred_ids)

In [ ]:
%%time
question = process_audio('question.wav')
question

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


CPU times: user 11.5 s, sys: 141 ms, total: 11.6 s
Wall time: 12.6 s


'як я можу створити унікальний номер з цифир які мені подобаються'

# RAG

In [ ]:
embd_model_e5 = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

<ipython-input-6-6763f7661b4b>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embd_model_e5 = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")


In [ ]:
embeddings_e5 = []

for question in tqdm(questions):
  embeddings_e5.append( embd_model_e5.embed_query(question) )

embeddings_e5 = np.float32(embeddings_e5)

100%|██████████| 299/299 [00:08<00:00, 36.88it/s]


In [ ]:
db = faiss.IndexFlatL2(embeddings_e5.shape[1])
db.add(embeddings_e5)

## Rag

### Set up LLM

In [ ]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    token=userdata.get("HF_TOKEN")
)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
model.to("cuda")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    skip_special_tokens=False,
    cache_dir="./cache/",
    token=userdata.get("HF_TOKEN")
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
text_generation_pipeline = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    return_full_text=True,
    max_new_tokens = 400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

<ipython-input-16-4e63169d82dc>:9: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
prompt_template = """<|begin_of_text|><<SYS>>
Ти – асистент бот у компанії Київстар. Твоє завдання – відповідати на запитання, що стосуються зв'язку та інтернету, виключно на основі наданого контексту.

ПРАВИЛА:
* Відповідай тільки тоді, коли питання має прямий зв’язок із наданим контекстом і контекст містить необхідну інформацію для відповіді.
* Не відповідай, якщо контекст не має відповідної інформації чи не стосується питання.
* Уникай спекуляцій або додавання деталей поза контекстом. Відповідь повинна чітко відповідати на питання на основі контексту.
* Якщо контекст не надано або не містить відповідної інформації, відповідай "Я не маю відповіді на це запитання".

Пам'ятай: відповідай тільки тоді, коли контекст чітко надає інформацію для цього.

Контекст:
{context}
<</SYS>>
<|eot_id|>
<<USER>>
[INST] Питання: {question} [/INST]
<</USER>>
<|eot_id|>
<<ASSISTANT>>
Відповідь: """

In [ ]:
prompt = PromptTemplate(
  template = prompt_template,
  input_variables = ["context", "question"]
)

In [ ]:
llm_chain = prompt | llm | StrOutputParser()

In [ ]:
def get_context_elements(query):
  query_embedding = np.float32([embd_model_e5.embed_query(query)])
  # Perform the search
  D, I = db.search(query_embedding, k=5)  # k is the number of nearest neighbors
  return questions[I][0]

def transform_context(context):
  string = ""

  for i, doc in enumerate(context, start=1):
    string += f"Уривок #{i}: \"{doc}\"\n\n"

  return string.strip()

retriever = lambda x: transform_context(get_context_elements(x))

In [ ]:
rag_chain = (
  {"context": retriever, "question": RunnablePassthrough()} | llm_chain
)

`question` is taken from Speach to text

In [ ]:
def get_completion(query: str):
  result = rag_chain.invoke(query)
  output_text = result.split("<<ASSISTANT>>\nВідповідь: ")[1].split("<|user|>")[0].strip()
  return output_text, 0, 0

# Text to Speech

In [ ]:
tts = TTS(device="cuda")

downloading https://github.com/robinhad/ukrainian-tts/releases/download/v6.0.0
Found ./model.pth. Skipping download...
Found ./config.yaml. Skipping download...
Found ./spk_xvector.ark. Skipping download...
Found ./feats_stats.npz. Skipping download...
downloaded.


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/usr/local/lib/python3.10/dist-packages/espnet2/tasks/abs_task.py:1857: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `w

In [ ]:
def output_to_voice(output_text: str, output_file: str = "reponse.wav"):
  with open(output_file, mode="wb") as file:
    _, output_text = tts.tts(output_text, Voices.Dmytro.value, Stress.Dictionary.value, file)

  # set speed x1.2
  speed_factor = 1.2
  sampling_rate, data = wavfile.read(output_file)
  new_sampling_rate = int(sampling_rate * speed_factor)
  wavfile.write(output_file, new_sampling_rate, data)

  return output_text

# Final

In [ ]:
def all_together(input_file: str, output_file: str = "reponse.wav"):
  start = time.time()

  question = process_audio(input_file)
  step1 = time.time()

  output_text, input_price, output_price = get_completion(question)
  step2 = time.time()

  total_price = input_price + output_price

  accented_text = output_to_voice(output_text, output_file)
  step3 = time.time()

  times = {
      "stt": step1 - start,
      "llm": step2 - step1,
      "tts": step3 - step2,
      "total": step3 - start
  }

  return question, output_text, total_price, times

In [ ]:
%%time
question, output_text, total_price, times = all_together("question.wav", "reponse.wav")

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


як я можу створити унікальний номер з цифир які мені подобаються


/usr/local/lib/python3.10/dist-packages/espnet2/torch_utils/device_funcs.py:29: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return to_device(torch.from_numpy(data), device, dtype, non_blocking, copy)


RTF = 0.167748
CPU times: user 9.23 s, sys: 120 ms, total: 9.35 s
Wall time: 9.56 s


In [ ]:
print(question)

як я можу створити унікальний номер з цифир які мені подобаються


In [ ]:
print(output_text)

Скористайтеся послугою Ексклюзивний номер. Підключити її можна в нашому інтернет-магазині, найближчому магазині Київстар або магазинах партнерів. Деталі – в описі послуги Ексклюзивний номер. <</ASSISTANT>>


In [ ]:
f"${total_price:.6f}"

'$0.000000'

In [ ]:
print(times)

{'stt': 0.04455161094665527, 'llm': 6.619278907775879, 'tts': 2.897444248199463, 'total': 9.561274766921997}


## Generate questions

In [ ]:
ipd.Audio(filename="reponse.wav")

In [ ]:
!mkdir audios

In [ ]:
import time

In [ ]:
times = []

with open("modified_questions.json", "r") as f:
  modified_questions = json.load(f)

  for i, question in tqdm(enumerate(modified_questions["questions"])):
    original = question["original"]
    modified = question["modified"]

    start = time.time()
    output_to_voice(modified, f"./audios/question_{i}.wav")
    times.append(time.time() - start)

In [ ]:
!zip -r audios.zip audios

In [ ]:
np.mean(times)

0.5902021445480048

In [ ]:
np.std(times)

0.1144984802447464

## Test all system

In [ ]:
!mkdir llama_responses

In [ ]:
from glob import glob

In [ ]:
wavs = list(glob("./audios/question_*.wav"))
wavs = list(sorted(wavs, key=lambda x: int(x.split("_")[-1].split(".")[0])))

sst_times = []
tts_times = []
llm_times = []
total_times = []


for i, audio_file in tqdm(enumerate(wavs), total=len(wavs)):
  print(i, audio_file)

  question, output_text, total_price, times = all_together(
    audio_file,
    f"./llama_responses/response_{i}.wav"
  )

  sst_times.append(times['stt'])
  tts_times.append(times['tts'])
  llm_times.append(times['llm'])
  total_times.append(times['total'])

  with open(f"./llama_responses/response_{i}.txt", 'w') as f:
    f.write(output_text)

  0%|          | 0/51 [00:00<?, ?it/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


0 ./audios/question_0.wav


  2%|▏         | 1/51 [00:04<03:31,  4.23s/it]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


RTF = 0.126898
1 ./audios/question_1.wav


  4%|▍         | 2/51 [00:08<03:20,  4.10s/it]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

RTF = 0.148488
49 ./audios/question_49.wav


 98%|█████████▊| 50/51 [08:29<00:07,  7.74s/it]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


RTF = 0.130752
50 ./audios/question_50.wav


100%|██████████| 51/51 [08:38<00:00, 10.16s/it]

RTF = 0.129886


In [ ]:
np.mean(sst_times), np.std(sst_times)

(0.04266776758081773, 0.007818094807109707)

In [ ]:
np.mean(llm_times), np.std(llm_times)

(7.708785346910065, 6.669677050769595)

In [ ]:
np.mean(tts_times), np.std(tts_times)

(2.4119862621905757, 2.143897027950164)

In [ ]:
np.mean(total_times), np.std(total_times)

(10.163439376681458, 8.807301590910315)

In [ ]:
!zip -r llama_responses.zip llama_responses